### Use ``Open Route Service API`` to retrieve distance matrix
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
from add_distance import add_distance_time
import openrouteservice as ors

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2014]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude'])
print(place_df.shape)
place_df.head(10)

(7025, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
2295,TAHARA BRIDGE RESERVE PICNIC AREA,-37.66568,141.66459,2014,park,217011420
2296,CULGOA RECREATION RESERVE,-35.71682,143.10101,2014,park,215031400
2297,CUMBERLAND,-38.57263,143.94681,2014,park,203031049
2298,WAREENA PARK,-36.35911,146.30977,2014,park,204021066
2299,ENTERPRIZE PARK,-37.82000,144.95973,2014,park,206041505
2300,RYANS RESERVE,-37.66807,145.15741,2014,park,209031215
2301,FERNTREE GULLY PICNIC GROUND,-37.89088,145.31692,2014,park,211051282
2302,LORRAINE KOVACS RESERVE,-37.97763,145.25403,2014,park,212021454
2303,RAY BASTIN RESERVE,-38.03359,145.30791,2014,park,212021456
2304,NEVILLE HAMILTON RESERVE,-38.02612,145.34533,2014,park,212021293


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2014_property_with_SA2.csv')[PROPERTY_COLS]
# 2013 example
print(property_df.shape)
property_df.head(10)

(12489, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027
1,"36 ALBERT STREET, CHEWTON",-37.093028,144.259469,3.0,1.0,1,320,House,3451,2014,8,House,202021027
2,"14 PITMAN STREET, CHEWTON",-37.083013,144.266589,4.0,1.0,1,345,House,3451,2014,1,House,202021027
3,"52 FRYERS ROAD, CHEWTON",-37.087738,144.257555,3.0,1.0,2,420,House,3451,2014,2,House,202021027
4,"114 FRYERS ROAD, CHEWTON",-37.094488,144.255400,2.0,1.0,3,260,House,3451,2014,9,House,202021027
5,"57 ARCHERS ROAD, CHEWTON",-37.094426,144.259118,3.0,1.0,2,395,House,3451,2014,5,House,202021027
6,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,270,House,3451,2014,10,House,202021027
7,"732A PYRENEES HIGHWAY, CHEWTON",-37.083670,144.272154,2.0,1.0,1,205,House,3451,2014,7,House,202021027
8,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,200,House,3683,2014,1,House,204031070
9,"2/20 NICKLESS STREET, CHILTERN",-36.154038,146.602977,2.0,1.0,2,190,Rental_residential,3683,2014,3,Other,204031070


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(merged_df.shape)
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

(195828, 18)
Distinct sa2 codes = 483


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,OLD DIAMOND HILL RESERVE,-37.08179,144.20126,park
1,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE,-37.06316,144.21379,station
2,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CHEWTON,-37.08485,144.25848,station
3,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
4,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary
5,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE PRIMARY SCHOOL,-37.06660,144.22233,primary
6,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CHEWTON PRIMARY SCHOOL,-37.08233,144.26116,primary
7,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CAMPBELLS CREEK PRIMARY SCHOOL,-37.09293,144.20466,primary
8,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,ST MARYS SCHOOL,-37.06396,144.22096,primary
9,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary


In [16]:
subset = merged_df[merged_df['SA2_CODE'] == 201011002].reset_index(drop=True)
count_places = len(subset.drop_duplicates(subset=['Place_Names','latitude_des']))
print(f"Subset size = {len(subset)}, Places count = {count_places}, Property count = {len(subset) // count_places}")
subset.head(60)

Subset size = 1659, Places count = 21, Property count = 79


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,VICTORIA PARK,-37.56447,143.82385,park
1,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,PRINCE OF WALES PARK,-37.54973,143.81711,park
2,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,PRINCE OF WALES RECREATION RESERVE,-37.54973,143.81711,park
3,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,ALFRED DEAKIN PLACE,-37.56076,143.85890,park
4,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,ALBERT H GRAHAM RESERVE,-37.56513,143.86300,park
5,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,CITY OVAL,-37.55832,143.83778,park
6,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,BALLARAT BOTANICAL GARDENS,-37.54861,143.82138,park
7,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,LINTON JUNCTION,-37.53991,143.82115,station
8,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,WENDOUREE,-37.53978,143.82003,station
9,"2/305 PLEASANT STREET, BALLARAT CENTRAL",-37.566758,143.837738,2.0,2.0,2,310,Rental_residential,3350,2014,8,Other,201011002,Urquhart Park Primary School,-37.56600,143.83122,primary


In [18]:
client = ors.Client('5b3ce3597851110001cf62482d289033372149f7b6419d839d18a3dc') # gracelovesyah api key 500 

#### Specify the ``year`` here

In [19]:
#added_distance_merged_df = add_distance_time(merged_df, 2013) # specify the year here
added_distance_merged_df = add_distance_time(merged_df, 2014, client, 'testing')

Executing SA2 Code 202021027
Subset size = 476, Places count = 14, Property count = 34
Quota Exceeded daily limit or (less than 3500 branch) timeout
Executing SA2 Code 204031070
Subset size = 144, Places count = 9, Property count = 16
To Place Distance Grand List, Normal Branch, length = 620
Executing SA2 Code 210021235
Subset size = 120, Places count = 10, Property count = 12
To Place Distance Grand List, Normal Branch, length = 740
Executing SA2 Code 211051275
Subset size = 39, Places count = 3, Property count = 13
To Place Distance Grand List, Normal Branch, length = 779
Executing SA2 Code 209031212
Subset size = 598, Places count = 26, Property count = 23
To Place Distance Grand List, Normal Branch, length = 1377
Executing SA2 Code 211051276
Subset size = 440, Places count = 22, Property count = 20
To Place Distance Grand List, Normal Branch, length = 1817
Executing SA2 Code 205041094
Subset size = 1326, Places count = 39, Property count = 34
To Place Distance Grand List, Normal Br

In [14]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.82098,park,29315.38,29.31538,1608.83,26.813833,97567.05,97.56705,5153.58,85.893
1,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.64581,park,2083.04,2.08304,204.11,3.401833,97567.05,97.56705,5153.58,85.893
2,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.71070,park,40137.92,40.13792,2374.66,39.577667,97567.05,97.56705,5153.58,85.893
3,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.84042,park,31056.38,31.05638,1675.11,27.918500,97567.05,97.56705,5153.58,85.893
4,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.83459,park,30089.42,30.08942,1813.06,30.217667,97567.05,97.56705,5153.58,85.893
5,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.76368,park,13669.72,13.66972,839.59,13.993167,97567.05,97.56705,5153.58,85.893
6,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.84042,park,21342.96,21.34296,1294.27,21.571167,97567.05,97.56705,5153.58,85.893
7,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.84959,park,32063.80,32.06380,1715.40,28.590000,97567.05,97.56705,5153.58,85.893
8,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.67653,park,4693.52,4.69352,482.38,8.039667,97567.05,97.56705,5153.58,85.893
9,"75 FORREST DRIVE, NYORA",-38.305928,145.652314,3.0,1.0,6,350,House,3987,2014,...,145.82903,park,29453.14,29.45314,1613.10,26.885000,97567.05,97.56705,5153.58,85.893


In [15]:
added_distance_merged_df.to_csv('test_distance.csv', index=False)